# Generating data

This notebook is used to generate data inside the web application.

Watch out: it immediately saves data on the database!

Setting-up: https://stackoverflow.com/questions/35483328/how-to-setup-jupyter-ipython-notebook-for-django

In [2]:
import os
import sys
sys.path.append('../')
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "jmapp.settings")
django.setup()
from vote.models import *
from django.utils import timezone
from datetime import timedelta
from random import randint
import numpy as np
from math import ceil 

gs = Grade.objects.all()
vs = Voter.objects.all()
cs = Candidate.objects.all()

## 1. Create basic grades

In [2]:
grades = [("Très bien", "tb"),
             ("Bien", "b"),
             ("Assez bien", "ab"),
             ("Correct", "ok"),
             ("Insuffisant", "insuf"),
            ]

for n, c in grades:
    Grade(name=n, code=c).save()

## 2. Create fake candidates
It uses a python API to randomuser.me :
https://github.com/null-none/random-user-generation

In [11]:
from examples.generation import RandomUserGeneration
from django.contrib.auth.hashers import make_password
from datetime import date, datetime

from vote.models import Candidate, User

Ncandidates = 5

for i in range(Ncandidates - len(cs)):
    # create user and fake user
    user = User()
    ruser = RandomUserGeneration()
    
    user.email     = ruser.get_mail()
    user.first_name  = ruser.get_first_name()
    user.last_name   = ruser.get_last_name()
    user.username    = ruser.data['results'][0]['login']['username']
    user.password    = make_password(ruser.data['results'][0]['login']['password'])

    user.save()
    
    # create candidate
    c     = Candidate()
    c.user = user
    c.picture     = ruser.get_picture()
    c.bio         = "Causis im si mo an talium juvare cumque tandem. Hinc quia inde rei mihi nec quum ibi."
    c.city        = ruser.data['results'][0]['location']['city']
    c.street      = ruser.data['results'][0]['location']['street']
    c.state       = ruser.data['results'][0]['location']['state']
    c.postcode    = ruser.data['results'][0]['location']['postcode']
    dob           = datetime.strptime(ruser.data['results'][0]['dob'], '%Y-%m-%d %H:%M:%S')
    c.birth_date  = dob.strftime('%Y-%m-%d')

    c.save()
    
    print("Candidate created: %s, %s" % (user.username, ruser.data['results'][0]['login']['password']))

Candidate created: bigkoala687, lord
Candidate created: beautifulleopard534, beerman
Candidate created: redleopard501, hound
Candidate created: crazykoala514, billyboy
Candidate created: lazygoose768, midget


# 3. Create fake voters

In [9]:
from examples.generation import RandomUserGeneration
from django.contrib.auth.hashers import make_password
from datetime import date, datetime

from vote.models import Candidate, User

Nvoters = 100

for i in range(Nvoters - len(vs)):
    # create user and fake user
    user = User()
    ruser = RandomUserGeneration()
    
    user.email       = ruser.get_mail()
    user.first_name  = ruser.get_first_name()
    user.last_name   = ruser.get_last_name()
    user.username    = ruser.data['results'][0]['login']['username']
    user.password    = make_password(ruser.data['results'][0]['login']['password'])

    user.save()
    
    # create candidate
    v             = Voter()
    v.user        = user
    v.picture     = ruser.get_picture()
    v.bio         = "Causis im si mo an talium juvare cumque tandem. Hinc quia inde rei mihi nec quum ibi."
    v.city        = ruser.data['results'][0]['location']['city']
    v.street      = ruser.data['results'][0]['location']['street']
    v.state       = ruser.data['results'][0]['location']['state']
    v.postcode    = ruser.data['results'][0]['location']['postcode']
    dob           = datetime.strptime(ruser.data['results'][0]['dob'], '%Y-%m-%d %H:%M:%S')
    v.birth_date  = dob.strftime('%Y-%m-%d')

    v.save()
    
    print("Voter created: %s, %s" % (user.username, ruser.data['results'][0]['login']['password']))

Voter created: goldensnake787, 3030
Voter created: whiteduck245, forum
Voter created: bluelion904, mercury1
Voter created: brownsnake636, hardy
Voter created: beautifulfish981, stupid1
Voter created: heavyswan409, london1
Voter created: crazyrabbit804, django
Voter created: brownswan800, hornet
Voter created: redrabbit534, wwwwwwww
Voter created: bigostrich140, hermes
Voter created: silvergorilla774, taco
Voter created: brownkoala300, carrie
Voter created: goldenrabbit294, cynthia
Voter created: orangepanda313, comet
Voter created: orangefish799, burner
Voter created: whitebird631, jockey
Voter created: redostrich766, virgil
Voter created: greencat873, seymour
Voter created: silverleopard576, panther1
Voter created: redwolf890, sawyer
Voter created: ticklishsnake701, case
Voter created: redpeacock107, sealteam
Voter created: yellowpeacock565, football1
Voter created: crazytiger543, nnnnnnnn
Voter created: greenswan823, chicken
Voter created: beautifulostrich336, wrangler
Voter created:

## 4. Create fake votes

In [5]:
import itertools

for v, c in itertools.product(vs, cs):
    g = Grade.objects.order_by('?').first()
    Rating(voter=v, candidate=c, grade=g).save()